# YOLO-DETR benchmark

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

df = pd.read_csv("benchmark.csv")
os.makedirs("figures", exist_ok=True)

df['Precision'] = df['TP'] / (df['TP'] + df['FP'])
df['Recall'] = df['TP'] / (df['TP'] + df['FN'])
df['F1-Score'] = 2 * (df['Precision'] * df['Recall']) / (df['Precision'] + df['Recall'])

df['subgroup'] = df['model'].str[4:]
df['subgroup'][df["model"]=="rtdetr-x"] = "detr-x"
df['subgroup'][df["model"]=="rtdetr-l"] = "detr-l"
df["group"]=df["subgroup"].str[:2]
df['group'][df["model"]=="rtdetr-l"] = "detr"
df['group'][df["model"]=="rtdetr-x"] = "detr"
df['group'][df["subgroup"]=="v10x"] = "v10"

sub_df=df.loc[[9,13,14,16,15,7,8,17,18,23]].reset_index(drop=True) # models kept for the paper... trainings are not available in this code, but you can reproduce the results with detection.iypnb

In [ ]:
fig = plt.figure(figsize=(8, 5))

for idx, row in sub_df.iterrows():
    plt.bar(idx, row['FPS'], width=0.3, color='blue', label='FPS' if idx == 0 else "")

plt.xticks(range(len(sub_df)), sub_df['subgroup'], rotation=45, fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Features per second', fontsize=15)   
plt.grid(visible=True, alpha=0.3)
plt.ylim(6, 12)
plt.tight_layout()
plt.show()

pdf = PdfPages("figures/FPS.pdf")
pdf.savefig(fig, bbox_inches='tight')
pdf.close()

In [ ]:
labels = sub_df["model"].values  
y_labels = [0.2, 0.4, 0.6, 0.8]

# Normalize the angles for the spider plot
angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()

# Append first value to close the loop
angles += angles[:1]
recall = sub_df["Recall"].tolist() + [sub_df["Recall"].iloc[0]]
precision = sub_df["Precision"].tolist() + [sub_df["Precision"].iloc[0]]
f1_score = sub_df["F1-Score"].tolist() + [sub_df["F1-Score"].iloc[0]]

# Create spider plot
fig, ax = plt.subplots(figsize=(20, 20), subplot_kw=dict(polar=True))

# Plot each metric and close the loop
ax.plot(angles, recall, label="Recall", color="blue", linestyle='solid', linewidth=3)
ax.fill(angles, recall, color="blue", alpha=0.1)

ax.plot(angles, precision, label="Precision", color="red", linestyle='solid', linewidth=3)
ax.fill(angles, precision, color="red", alpha=0.1)

ax.plot(angles, f1_score, label="F1-Score", color="green", linestyle='solid', linewidth=3)
ax.fill(angles, f1_score, color="green", alpha=0.1)

# Adjust x-ticks (move further out)
ax.set_xticks(angles[:-1])  # Exclude the duplicate angle
ax.set_xticklabels(labels, fontsize=25, fontweight='bold', ha='center')

# Move x-ticks further from the circle
for label, angle in zip(ax.get_xticklabels(), angles[:-1]):
    label.set_horizontalalignment('center')
    label.set_verticalalignment('center')
    label.set_y(label.get_position()[1] - 0.09)  # Move the labels outward

ax.set_yticklabels(y_labels, fontsize=40)

# Move the legend further away from the plot
ax.legend(loc="upper right", bbox_to_anchor=(1.4, 1.2), fontsize=40, frameon=True)
plt.show()

from matplotlib.backends.backend_pdf import PdfPages
pdf = PdfPages("figures/spider_benchmark.pdf")
pdf.savefig(fig)
pdf.close()

# WC metric benchmark

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

df = pd.read_csv("wc_benchmark.csv")

df['Precision'] = df['TP'] / (df['TP'] + df['FP'])
df['Recall'] = df['TP'] / (df['TP'] + df['FN'])
df['F1-Score'] = 2 * (df['Precision'] * df['Recall']) / (df['Precision'] + df['Recall'])

In [ ]:
metrics = ['Recall', 'Precision', 'F1-Score']
colors = ['blue', 'red', 'green']

fig=plt.figure(figsize=(8, 5))

for idx, row in df.iterrows():
    plt.bar(idx - 0.20, row['Recall'], width=0.3, color='blue', label='Recall' if idx == 0 else "")
    plt.bar(idx, row['Precision'], width=0.3, color='red', label='Precision' if idx == 0 else "")
    plt.bar(idx + 0.20, row['F1-Score'], width=0.3, color='green', label='F1-Score' if idx == 0 else "")

plt.xticks(range(len(df)), df['WC'], rotation=45, fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Score', fontsize=16)
plt.grid(visible=True, alpha=0.4)
plt.legend(fontsize=15, loc='lower right', framealpha=1)
plt.tight_layout()
plt.show()
pdf = PdfPages("figures/wc_benchmark.pdf")
pdf.savefig(fig)
pdf.close()


# CLIP plots

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
all_stats = pd.read_csv("clip_stats.csv")
df = all_stats.copy()

# Step 1: Add 'argmax' column
p_columns = [col for col in df.columns if col.startswith("p")]
df['argmax'] = df[p_columns].idxmax(axis=1)

# Adding new features to the dataset
fruit_columns = ["p0", "p1", "p2", "p3", "p4", "p5"]
backg_columns = ["p6", "p7", "p8", "p9", "p10", "p11", "p12", "p13", "p14", "p15", "p16"]

df["fruit_mean"] = df[fruit_columns].mean(axis=1)
df["backg_mean"] = df[backg_columns].mean(axis=1)

# Adding argmax probabilities
sorted_probs = df.iloc[:, 1:18].apply(lambda row: row.nlargest(2).values, axis=1)
df["argmax_prob"] = sorted_probs.apply(lambda x: x[0])
df["argmax2_prob"] = sorted_probs.apply(lambda x: x[1])

# Step 2: Calculate normalized counts of 'argmax'
overall_argmax_counts = df['argmax'].value_counts(normalize=True)

# Step 3: Separate by WC and calculate normalized counts
wc_grouped = df.groupby('WC')
wc_argmax_counts = wc_grouped['argmax'].value_counts(normalize=True).unstack()

# Step 4: Average values globally and per weather condition
overall_averages = df[p_columns].mean()
wc_averages = wc_grouped[p_columns].mean()

# Step 5: Replace x-axis labels using prompt_list
prompt_list = [
    "a picture of an orange in a tree",
    "a picture of an unripe orange in a tree",
    "a picture of a part of an orange in a tree",
    "a picture of a part of an unripe orange in a tree",
    "a picture of a lemon in a tree",
    "a picture of an unripe lemon in a tree",
    "a picture of leaves and tree branches without any ripe or unripe fruit",
    "a picture of leaves and tree branches",
    "a picture of parts of leaves and tree branches",
    "a picture of parts of leaves and tree branches without any ripe or unripe fruit",
    "a picture of a building",
    "a picture of a part of a building",
    "a picture of a part of the sky",
    "a picture of dead leaves on the ground without any ripe or unripe fruit",
    "a picture of dead leaves on the ground",
    "a picture of the roots of a tree",
    "a picture of the roots of a tree without any ripe or unripe fruit"
]
p_column_mapping = {f"p{i}": prompt_list[i] for i in range(len(prompt_list))}
df.rename(columns=p_column_mapping, inplace=True)
overall_averages.rename(index=p_column_mapping, inplace=True)
wc_averages.rename(columns=p_column_mapping, inplace=True)
overall_argmax_counts.rename(index=p_column_mapping, inplace=True)
wc_argmax_counts.rename(columns=p_column_mapping, inplace=True)
avg_values_all = overall_averages.values
argmax_values_all = overall_argmax_counts.reindex(prompt_list, fill_value=0).values
x = np.arange(len(prompt_list))

fig = plt.figure(figsize=(8, 5))
width = 0.3 

# Barplots for averages and normalized argmax counts
plt.bar(x - width / 2, avg_values_all, width, label='Average probability value', color='lightblue')
plt.bar(x + width / 2, argmax_values_all, width, label='Normalized argmax count', color='orange')
plt.ylabel("Probability value", fontsize=15)
plt.xticks(ticks=x, labels=all_stats.columns[1:].tolist(), rotation=45, fontsize=15) # usually labels=prompt_list
plt.yticks(fontsize=13)
plt.legend(fontsize=15)
plt.tight_layout()
plt.show()

pdf = PdfPages("figures/CLIP_prompts.pdf")
pdf.savefig(fig)
pdf.close()

for wc, wc_data in wc_averages.iterrows():
    # Prepare data for combined plot
    avg_values = wc_data.values
    argmax_values = wc_argmax_counts.loc[wc].reindex(prompt_list, fill_value=0).values
    x = np.arange(len(prompt_list))

    # Plot
    plt.figure(figsize=(8, 5))
    width = 0.3  # Width of the bars

    # Barplots for averages and normalized argmax counts
    plt.bar(x - width / 2, avg_values, width, label='Average probability value', color='lightgreen', alpha=0.7)
    plt.bar(x + width / 2, argmax_values, width, label='Normalized argmax count', color='purple', alpha=0.7)

    # Titles, labels, and legends
    plt.title(f"CLIP - Average probability values and argmax counts for {wc}", fontsize=16, fontweight='bold')
    plt.ylabel("Probability value", fontsize=13)
    plt.xticks(ticks=x, labels=all_stats.columns[1:].tolist(), rotation=45, fontsize=15) # usually labels=prompt_list
    plt.yticks(fontsize=15)
    plt.legend(fontsize=15)
    plt.tight_layout()
    plt.show()

In [ ]:
# Count statistics grouped by WC
grouped_counts = df.groupby("WC").apply(
    lambda group: pd.Series({
        "total_samples": len(group),
        "fruit_mean_gt_backg_mean": (group["fruit_mean"] > group["backg_mean"]).sum(),
        "argmax_in_fruit": group["argmax"].isin(fruit_columns).sum()
    })
).reset_index()

# Plotting
x = np.arange(len(grouped_counts))  # x-axis positions
width = 0.25  # Width of bars

fig = plt.figure(figsize=(8, 5))

# Bar plots for the three counts
plt.bar(x - width, grouped_counts["total_samples"], width, label="Total number of samples", color="lightblue")
plt.bar(x, grouped_counts["fruit_mean_gt_backg_mean"], width, label="Fruit Mean > Backg Mean", color="orange")
plt.bar(x + width, grouped_counts["argmax_in_fruit"], width, label="Argmax among fruit prompts", color="green")

# Labels, titles, and legend
plt.xticks(ticks=x, labels=grouped_counts["WC"], fontsize=20)
plt.yticks(fontsize=15)
plt.ylabel("Counts", fontsize=18)
plt.legend(fontsize=16)
plt.tight_layout()
plt.show()
pdf = PdfPages("figures/CLIP_metrics.pdf")
pdf.savefig(fig)
pdf.close()

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

grouped_probs = df.groupby("WC").apply(
    lambda group: pd.Series({
        "avg_backg_mean": group["backg_mean"].mean(),
        "avg_fruit_mean": group["fruit_mean"].mean(),
        "avg_argmax_prob": group["argmax_prob"].mean(),
        "avg_argmax2_prob": group["argmax2_prob"].mean(),
        "prop_fruit_mean_gt_backg_mean": (group["fruit_mean"] > group["backg_mean"]).mean(),
        "prop_argmax_in_fruit": group["argmax"].isin(fruit_columns).mean()
    })
).reset_index()

x = np.arange(len(grouped_probs)) 
width = 0.20  

fig=plt.figure(figsize=(8, 5))

plt.bar(x - 2 * width, grouped_probs["avg_backg_mean"], width, label="Avg Backg Mean", color="blue")
plt.bar(x - width, grouped_probs["avg_fruit_mean"], width, label="Avg Fruit Mean", color="orange")
plt.bar(x, grouped_probs["avg_argmax_prob"], width, label="Avg Argmax Prob", color="green")
plt.bar(x + width, grouped_probs["avg_argmax2_prob"], width, label="Avg Argmax2 Prob", color="red")
plt.xticks(ticks=x, labels=grouped_probs["WC"], fontsize=17)
plt.yticks(fontsize=15)
plt.ylabel("Probability", fontsize=17)
plt.legend(fontsize=11, loc='upper right')
plt.tight_layout()
plt.show()

pdf = PdfPages("figures/CLIP_probs.pdf")
pdf.savefig(fig)
pdf.close()


# Statistics about bounding boxes

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from img_bbox import read_image, parse_annotations, get_cropped_boxes
ALL_WC_PATH = "FINAL_SUBIMAGES/"

def get_sub_max(all_folders_path):
    """Process all images and labels in the given folders."""
    shapes = pd.DataFrame(columns=["width", "height", "WC"])
    index = 0
    for folders in os.listdir(all_folders_path):
        if folders != ".DS_Store":
            img_folder_path = all_folders_path + folders + "/images/"
            label_folder = all_folders_path + folders + "/labels/"
            weather = folders[:2]
            for image_file in os.listdir(img_folder_path):
                if image_file.endswith(".jpg") or image_file.endswith(".jpeg") or image_file.endswith(".png"):
                    image_path = os.path.join(img_folder_path, image_file)
                    label_path = os.path.join(label_folder, os.path.splitext(image_file)[0] + '.txt')
                    if os.path.exists(label_path):
                        img_rgb = read_image(image_path)
                        annotations = parse_annotations(label_path, img_rgb.shape)
                        cropped_images = get_cropped_boxes(img_rgb, annotations)
                        for img in cropped_images:
                            height, width, _ = img.shape
                            shapes.loc[index] = [width, height, weather]
                            index += 1                  
    return shapes

shapes = get_sub_max(ALL_WC_PATH)

In [ ]:
fig = plt.figure(figsize=(8, 5))
xlim=80
hist = plt.hist2d(shapes['height'], shapes['width'], bins=[64, 64], range=[[0,xlim], [0,xlim]], cmap='viridis', rasterized=True)

cbar = plt.colorbar(hist[3])
cbar.set_label('Number of Instances', fontsize=15)
cbar.ax.tick_params(labelsize=13)
plt.xlabel('Height', fontsize=15)
plt.ylabel('Width', fontsize=15)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.grid(False)
plt.show()

pdf = PdfPages("figures/bbox_hist"+str(xlim)+".pdf")
pdf.savefig(fig)
pdf.close()